In [1]:
from PIL import Image, ImageOps, ImageFont
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets
from ipywidgets import interactive, HBox, VBox
from ascii_art_conversion_1x1 import img2char_arr_1x1
from ascii_art_conversion_1x2 import img2char_arr_1x2
from img_processing import preprocess_img, DITHER_MODES
from braille_art import img2braille_arr, get_braille_chars
from generate_ascii_palette import *
from ascii_art_conversion_mlp import train_classifier

In [2]:
ASCII_SET = get_asciis()
BRAILLE_SET = get_braille_chars()
ASCII_AND_BRAILLE_SET = ASCII_SET + BRAILLE_SET

In [3]:
FONT = ImageFont.truetype("fonts/CascadiaMono.ttf", 12)
IMG_PATH = "imgs/irad_grad.bmp"

In [4]:
orig_img = Image.open(IMG_PATH).convert("L")
#orig_img = ImageOps.invert(orig_img)

print(orig_img.size)

(512, 512)


In [5]:
def i_preprocess_img(scale_factor, contrast, brightness, quantize_colors, eq, dither):
    proc_img = preprocess_img(orig_img, 0.3, contrast, brightness, eq, quantize_colors, dither)
    plt.imshow(proc_img, cmap='gray', vmin=0, vmax=255, interpolation='none')

    plt.figure(figsize=(9, 3))
    plt.xticks([x for x in range(0, len(proc_img.histogram()), 25)])
    plt.bar([x for x in range(0, len(proc_img.histogram()))], proc_img.histogram())
    plt.grid()
    plt.show()

    return proc_img

interactive_preprocess = interactive(i_preprocess_img,
            scale_factor=(0.01, 1, 0.01),
            contrast=(0, 2, 0.01),
            brightness=(0, 2, 0.01),
            eq=(0, 1, 0.01),
            quantize_colors=(1, 256, 1),
            dither=DITHER_MODES)

controls = VBox(interactive_preprocess.children[:-1])
display(HBox((controls, interactive_preprocess.children[-1])))

In [6]:
proc_img = interactive_preprocess.result

In [7]:
palette, palette_br = generate_1_1_palette(ASCII_SET, FONT, 12, normalize=True)
ascii_arr = img2char_arr_1x1(proc_img, palette, palette_br, dither=DITHER_MODES.FS)
for i in range(len(ascii_arr)):
    for j in range(len(ascii_arr[i])):
        print(ascii_arr[i][j], sep='', end='')
    print('\n', end='')

         . ...........:.:.-.,_`-','-`-,'-!_`!:!,^,;!!:^;!;^!!!!^!;^^!;^^;^^!!;^;;^!;^^^^^!!;^^!!!^^;^';;-;,!-^-;'-'_-,-`,_-,-`.'.'.._......... . .        
    . ...........`.`-_._:'-_,`,`'_;_^`!^^`!^;^;;!;!^;!;^^^!!^^^^;;!"!"!~!~!"!~;"^~^";"!!"^;!^!^!^^^!;^^!!^^!^;,!;^!^;`!-!,,'-__''`'__`,.`............ .   
  . ..........'.:.`_.:,_-:',^``;';';_^^-!;^'!^;;^;^!;^;!!!^~;""~~"""""""~"~""~""~""~~~~~"~"""!~^!^^^^!^;;;^!;!!!_!-!:;_^_!,^:-!::`_''.':.'`,.:........ .  
 ..........,.,.'_''_,::``'^-,!';:;`!;'!!;^^!^;!;!;;!~^"~~~"~""~""""~"""~~""~"~~~~~"""~""""~~"""~~~""!~^~!!^;^;;;!!^!!!^,;:^_!,-^`_-_-',:,:.'.._.......... 
 . ......-.-.-__.`___`:;`_!'!-^:!;!^!^;;^^!;!;;;~!~~""~""~""""~~~"""~""""<~~<~~+~~+~"""~~~~~""""~"~"~~"~"~;~!^^^;^^!!,;;!^,!:;`;`^--:-`'``',:_.:.'..... ..
........`.-.`:.'``_:-;,,;`^-!^!^;;!!^!^;^!!""~~~"~~"~""~"~~~L"~?~>"="?<<"\"+"<~\~?"L~?/"="L~/~"\"~~""~~"~""~~!";~!^!^^!^!^;!^!`!`!:!`!-':,,.,.'_.-........
 ....'.'.:',----__:^_'!`^!:;;`^!!`^^^;^^~""~"~"""~~"~~<~+"L~<">~L"\"+~

In [8]:
palette, palette_br = generate_1_2_palette(ASCII_SET, FONT, (8,8), normalize=True)
ascii_arr = img2char_arr_1x2(proc_img, palette, palette_br, dither=DITHER_MODES.FS)
for i in range(len(ascii_arr)):
    for j in range(len(ascii_arr[i])):
        print(ascii_arr[i][j], sep='', end='')
    print('\n', end='')

                . : : :::::::::::::::::::';';-!:~~~-~!!!!!!!~~~!~~!!~!!!~~!~!!!!~~~~~!~!~~!~~~!~~~~~!!~!!-~:~:';-::::::::::::::::::: : : :                
          : : :: ::::::::::::::-:'-!'~'~!!-!!~~~~~~!~!!~~~!~!~~~!~!~!!!~~~+~~+!!+~~!~!!!~!!!~!~!~!~!!!~~!!!!!!!~~~-!-~--':'::::::::::::: ::: : :          
       . : : : ::: ::::::::-'--;-~:!'!!'!!!~~!~~~!~!~!~~~!+~+!+~+!+~+!+++!+!+!+!+~+++~+!+~+~+~+!~~~~!!!~!~!~~~!'!!!!-!~!-!''--:-::::::::: : : : : .       
     .  : : ::: ::::::::-:';';-~'!-!!!~!~~!!~!~!~~~+~+~+!+!+~+~+!+!+++~+~+++++++!>+!+++!++~+~+~+!+!+!+!!~~!!~!!~!~~~!~-~!'!;!';'-:-::::::::::: : : :      
   .  : : :: ::::::::':':-!-~-~!~~!~!~~~!!~~!~+!+!+!+~+!+~++++++<+>++\<+/=>+*+*>\+/>++\<+*+++++++!+~+!+!+!+~~~!!!~~~!!~~~!-~-~'~:-:-:-::::: : :: :  : :   
 .  : : :: :::::::':':-~'!'!-!!'!!~!!!~!~!+!+~!+~+~++++++/++><<>/\\*\\**\/*</\<=/==<*<*\>=/\>>+!/\+++++!+~+!+!+!~~!~!!!~~!!~~~-~'!'!:-:'::::::::: : :  :  
  : : :: ::::::::-:-;-!-!!~!~!~~!~~~~!+!+~!+~++!+++~/+=/\>>/=/>*///\</

In [9]:
ascii_arr = img2braille_arr(proc_img, dither=DITHER_MODES.FS)
for i in range(len(ascii_arr)):
    for j in range(len(ascii_arr[i])):
        print(ascii_arr[i][j], sep='', end='')
    print('\n', end='')

⠀⠀⠀⠀⠀⠀⡀⠀⠄⠠⠀⠄⡐⠠⠐⡀⢂⢐⠐⡐⢐⢐⢐⢐⢐⢐⠔⡐⢔⢐⠔⡰⠨⡐⢔⠨⡐⠔⠌⢔⢐⠌⢔⠰⡐⠌⢔⢐⠔⡐⠔⡐⡐⡐⡐⡐⡐⡐⢐⢀⢂⢐⠀⡂⠄⠄⠄⠠⠀⠠⠀⢀⠀⠀⠀⠀⠀
⠀⠀⠀⠄⠂⠁⠀⢂⠐⡀⠅⢂⠐⡈⡐⠠⠡⢐⠨⠠⡁⡂⡂⡢⢂⠅⡢⢊⠔⡐⢅⠢⡑⢌⠢⡑⢌⠪⡘⢔⠡⡊⡢⡑⢌⠜⡐⠔⢌⠢⡑⠌⢔⠨⡐⡐⡐⠄⠅⡂⠔⡀⠢⠐⡐⠐⡈⠠⠈⠄⠂⡀⠄⠂⠀⠄⠀
⠀⠀⠂⠠⠐⠈⡀⢂⠐⡀⢂⠂⡂⢂⠂⢅⢑⠐⢌⠂⢆⢊⠔⡨⢂⠕⡨⢂⠕⢌⢢⠱⡘⢔⢑⢌⢆⠣⡊⢆⠕⡌⡢⡊⡢⡑⠜⢌⢢⢑⠌⡌⡢⢑⢐⠌⢔⠡⡑⠄⢅⠂⠅⠅⡂⠡⠐⡈⢐⠀⡁⡀⠄⠐⢀⠠⠀
⠀⡈⢀⠂⠄⠁⠄⢂⠐⡐⢐⢐⠨⢐⢈⢂⠢⠡⡑⢌⠢⠢⡑⢌⢢⢑⢌⢆⢣⠱⡨⢪⢘⢌⠪⡢⡑⡅⢇⢕⢱⢘⢔⢱⠨⡊⢎⢊⠆⡕⢌⢆⠪⡨⠢⡡⡑⢔⠨⡨⠂⠅⠅⢅⠂⢅⠡⢐⠐⢐⠠⢀⠐⠈⡀⢀⠀
⠀⠄⡀⢂⠨⠀⠅⡂⠌⡐⡐⡐⠌⡂⡢⢑⠌⡪⢐⠅⢕⠅⡕⡑⡌⢆⢕⠔⡅⡇⡕⢕⢅⢇⢣⢱⢡⢣⢣⢱⢡⢣⢱⢸⢘⢜⠜⡌⢎⠜⡔⢜⢌⢪⠨⡢⢊⠢⡑⢔⠡⡃⠕⠡⡨⢐⠨⢐⠈⠄⡂⢐⠈⠄⠠⠀⠄
⠀⢂⠐⡀⢂⠡⢁⠄⠅⡂⠢⠨⡂⠪⡐⡡⢊⠔⢅⠕⡅⡕⢜⢌⢪⢊⢆⢇⢇⢎⢎⢎⢆⢇⢇⢇⢇⢇⢕⢕⢕⢕⢕⢱⢑⢕⢕⢕⠕⡕⡱⡑⡌⡆⢇⢪⠪⡘⢌⠢⡑⢌⠜⡨⡐⠔⡁⡂⠅⠅⡂⠂⠌⠠⠁⠌⠀
⠈⡀⢂⢐⠐⡈⠄⢌⢂⢊⠌⡪⢐⢑⢌⠢⡑⡅⡣⡱⡨⡪⢢⢣⢱⢑⢕⢜⢔⢕⢕⢕⢕⢕⢕⢕⢕⢕⢕⡕⡕⡕⡕⣕⢕⢕⢕⢕⢕⢕⢕⢕⢕⢱⢑⢅⢇⠕⡅⡣⡑⢅⠪⡐⢌⢂⠪⠠⡡⢁⢂⠅⠅⠡⠨⠀⠅
⠐⢐⠐⡀⡊⠄⡑⡐⢔⢐⠅⡪⡐⡱⡐⢕⢑⢌⢎⢢⢣⢪⢪⠪⡪⡪⡪⡪⣪⢪⢎⢮⢪⡺⡸⡱⣕⢽⢸⢜⢎⢇⡏⣎⢮⢺⢸⢜⢜⢜⢜⢔⢕⢕⢕⢕⢜⢸⢨⠢⡣⡑⡅⢕⠡⡊⢌⢊⠄⠕⡐⠄⢅⢑⠈⠄⡁
⠈⠄⡂⡂⡂⡑⡐⢌⢂⠆⢕⠰⡨⢢⢊⢎⢜⠔⡕⡅⡇⡕⡕⡕⣕⢕⢵⢹⡸⡜⣎⢮⢣⡳⣹⢪⡺⣜⢕⢗⢭⣣⢳⡱⡕⡧⡳⡱⡝⡜⡎⡇⡇⡇⡇⡇⡕⡕⡅⡇⡎⡌⢎⢌⢪⠨⡢⢑⠌⡊⠄⠕⡐⡐⠨⢐⠠
⠨⢐⢐⢐⢐⠌⡂⢕⢐⠅⢕⢑⠜⡌⢆⢣⢪⠪⡪⡪⡪⡪⡪⡎⡮⡺⣸⢱⢕⢧⢳⢕⡗⡽⣸⢕⢧⡳⣝⢝⢮⣪⡳⣹⡪⣳⢹⡪⡳⡹⡜⣎⢇⢗⢕⡕⡕⡕⡕⡕⡜⢜⢌⢪⠢⡱⢨⠢⡑⢌⠪⠨⡐⠨⠨⢐⠠
⠨⢐⢐⢐⢐⠌⡢⢑⠔⡑⡅⡣⡱⡸⡸⡸⡰⡹⡸⡸⡸⡸⣪⢺⢜⢮⣪⢳⢝⢮⢳⡳⣹⢝⢮⣫⡳⣝⢮⣫⡳⡵⣝⢮⣺⣪⡳⣝⢮⡫⡺⡜⣎⢧⢣⢇⢗⢕⢕⢕⢜⢜⢌⢆⠇⡎⡢⡱⠨⡢⢑⠅⡊⠌⢌⠐⠄
⠨⢐⢐⢐⠡⡊⢔⢑⢌⠪⡢⡱⡸⢰⢱⢸⢸⢸⢸⡸⡪⡣⣇⢗⢽⢜⡎⣗⢽⢕⡯⣺⢵⣫⢗⣗⡽⡮⣗⣗⡽⣝⢮⣳⣣⢗⣝⢮⣺⡪⡯⡺⡪⣎⢧⢳⡱⡣⣣⢣⢣⢣⢣⢱⢑⢕⢌⢪⠨⡢⡑⢌⠢⡑⡁⡊⠌
⠨⡐⡐⡡⢊⠔⡡⢢⢑⠕⡌⡆⢇⢇⢇⢇⢇⢧⢣⢇⢯⢺⡸⡵⡝⣮⡺⣕⢯⣳⢝⡷⣝⡮⣟⣞⣞⣽⣺⣺⣺⢽⣝⣞⢮⢯⣞⡵⣳⢝⡮⣫⢝⢮⡪⣇⢗⢝⡜⡜⡎⡎⡎⡎

In [10]:
from ascii_art_conversion_mxn import quantize_grayscale_mxn

brightness_shape = (2,4)
#char_to_brightness_map = find_brightness_map(ASCII_SET, FONT, brightness_shape, normalize=False)
char_to_brightness_map = generate_brightness_map(ASCII_SET, FONT, brightness_shape, normalize=True)
ascii_arr = quantize_grayscale_mxn(proc_img, char_to_brightness_map, (brightness_shape[1], brightness_shape[0]), DITHER_MODES.FS, 0.0)
for i in range(len(ascii_arr)):
    for j in range(len(ascii_arr[i])):
        print(ascii_arr[i][j], sep='', end='')
    print('\n', end='')

   ...::::::::;|;|||||[|[[[[[[[[[[[[[[[[[[[[[[[[[[[[[|[|||||;|:::::::::...  
 ...::::::;|;||||[|[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[|[|||;|:::::::...
..::::::;|:|||[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[|[|||;|::::::.
::::::;||||[|[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[|[||;|:;::::
::::;||;|[|[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[|[||:|:::
::;|:||[|[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[|[||;|::
:|:|||[|[[[[[[[[[[[[[[[[[[[[[[[[[[[[$[$[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[|||;:
:|||[|[[[[[[[[[[[[[[[[[[[[[[$[$[$[$[[$[$[$[$[$[$[[[[[[[[[[[[[[[[[[[[[[[[|[||
|;|[|[[[[[[[[[[[[[[[[[[[$[$[[$[$[$[$$[$[$[$[$[$[$[$[[[[[[[[[[[[[[[[[[[[[[|||
||[|[[[[[[[[[[[[[[[[[[$[[$[$[$$[$$$[$$$$$$$$$$[$[$[$[$[[[[[[[[[[[[[[[[[[[[[|
|[[[[[[[[[[[[[[[[[[[$[[$[$[$$[$$$$$$$$$$$$$$$$$$$$[$[$[$[[[[[[[[[[[[[[[[[[[|
|[[[[[[[[[[[[[[[[[$[[$[$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$[$[$[$[[[[[[[[[[[[[[[[[
[[[[[[[[[[[[[[[[[[[$[$$[$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$[$[$[[[[[[[[[[[[[[[[[[

In [11]:
brightness_shape = (2,4)
noise = 0.4
cls, char_to_brightness_map = train_classifier(ASCII_SET, FONT, brightness_shape, (32), 100, noise)

brightness_X = np.array([b.flatten() for b in char_to_brightness_map.values()])
brightness_y = list(char_to_brightness_map.keys())

repetitions = 50

test_X = np.tile(brightness_X, (repetitions,1))
test_y = list(char_to_brightness_map.keys()) * repetitions
test_X += ((np.random.random(test_X.shape)-0.5) * noise)
test_X = np.clip(test_X, a_min=0, a_max=1)

print('classifier score no noise', cls.score(brightness_X, brightness_y))
print('classifier score noise', cls.score(test_X, test_y))

ascii_arr = quantize_grayscale_mxn(proc_img, char_to_brightness_map, (brightness_shape[1], brightness_shape[0]), DITHER_MODES.FS, cls=cls)
for i in range(len(ascii_arr)):
    for j in range(len(ascii_arr[i])):
        print(ascii_arr[i][j], sep='', end='')
    print('\n', end='')

classifier score no noise 0.9578947368421052
classifier score noise 0.3717894736842105
       ..:::;:;|;|||||[|[[[[[[[[[[[[[[[[[[[[[[[[[[[[[|[||||;|;;:;::...      
    ..:::;;|;|||||[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[|[||||;|;::::..   
  ..::;:;|;||||[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[|||;|;;;:::.. 
 .:::;:|;|||[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[|[|||;|;:::.
.::;;|;|||[[|[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[|[||;|;:;:
:;:|;|||[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[|[||;|:;
:;|;||[|[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[|||;|:
;||||[|[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[||;|
|;|[|[[[[[[[[[[[[[[[[[[[[[[[[[[[$[$[$[$[$[$[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[||;
|||[[[[[[[[[[[[[[[[[[[[[[[[$[$[[[[[$[$[$[[[$[$[$[[[[[[[[[[[[[[[[[[[[[[[[[[[|
|[|[[[[[[[[[[[[[[[[[[[[[[$[[[[$[$$[$[$[$[$[$[$[[$[$[[[[[[[[[[[[[[[[[[[[[[[[|
|[[[[[[[[[[[[[[[[[[[[[[$[[$[$[$[$[$$$$$$$$$[$[$[$[$[$[[[[[[[[[[[[[